In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
query = """
SELECT 
*
FROM `perceptive-ivy-290216.nba_matchups_rollup.matchups_regs_23_24`
"""
project_id = "perceptive-ivy-290216"
matchups_totals = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
query = """
SELECT PLAYER_NAME, TEAM_ABBREVIATION, SEASON FROM `perceptive-ivy-290216.player_team_info.player_bios_regs_totals`
"""
project_id = "perceptive-ivy-290216"
player_team = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [4]:
def team_color(team):
    if team =="ATL":
        return "#C8102E"
    elif team =="DAL":
        return '#0050B5'
    elif team =="MEM":
        return '#0C2340'
    elif team =="MIA":
        return '#862633'
    elif team =="DEN":
        return '#1d428a'
    elif team =="DET":
        return '#1D4289'
    elif team =="GSW":
        return '#1D4289'
    elif team =="HOU":
        return '#BA0C2F'
    elif team =="IND":
        return '#041E42'
    elif team =="LAC":
        return '#0C2340'
    elif team =="LAL":
        return '#6a00ed'
    elif team =="POR":
        return '#C8102E'
    elif team =="PHX":
        return '#e56020'
    elif team =="PHI":
        return '#1D4289'
    elif team =="ORL":
        return '#0057B7'
    elif team =="OKC":
        return '#0072CE'
    elif team =="NYK":
        return '#FF8200'
    elif team =="NOP":
        return '#85714d'
    elif team =="MIN":
        return '#236192'
    elif team =="MIL":
        return '#2C5234'
    elif team =="BKN":
        return '#707372'
    elif team =="BOS":
        return '#007A33'
    elif team =="SEA":
        return '#00653A'
    elif team =="CHA":
        return '#00778B'
    elif team =="BUF":
        return '#418FDE'
    elif team =="CHI":
        return '#BA0C2F'       
    elif team =="CLE":
        return '#6F263D'
    elif team =="WAS":
        return '#C8102E'
    elif team =="UTA":
        return '#0C2340' 
    elif team =="TOR":
        return '#C8102E'
    elif team =="SAS":
        return '#9EA2A2'
    elif team =="SAC":
        return '#965ddd'                  
    
player_team['color'] = player_team['TEAM_ABBREVIATION'].apply(team_color)

In [5]:
year='2023-24'

In [8]:
#Combine player team data with player aggregated data so that we can color based on team
matchup_totals_year=matchups_totals.merge(player_team[player_team["SEASON"]==year], left_on=["DEF_PLAYER_NAME"], right_on=["PLAYER_NAME"])
player_color_dict_shot_dist=dict(zip(matchup_totals_year['PLAYER_NAME'], matchup_totals_year['color']))
team_color_dict_shot_dist=dict(zip(matchup_totals_year['TEAM_ABBREVIATION'], matchup_totals_year['color']))
matchup_totals_year.tail()

,SEASON_ID,POSITION,PERCENT_OF_TIME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,MATCHUP_AST,MATCHUP_TOV,MATCHUP_BLK,MATCHUP_FGM,MATCHUP_FGA,MATCHUP_FG_PCT,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,MATCHUP_FTM,MATCHUP_FTA,SFL,PLAYER_NAME,TEAM_ABBREVIATION,SEASON,color
2278,22023,F,0.450042,1628389,Bam Adebayo,71,423:27,2146.6,520,2361,140,70,23,214,441,0.485,44,134,0.328,48,64,35,Bam Adebayo,MIA,2023-24,#862633
2279,22023,F,0.530611,1629023,P.J. Washington,73,465:02,2381.7,574,2526,81,56,20,229,446,0.513,58,134,0.433,58,80,42,P.J. Washington,DAL,2023-24,#0050B5
2280,22023,F,0.423233,203999,Nikola Jokic,78,439:53,2339.8,795,2527,155,53,20,347,663,0.523,49,134,0.366,52,64,31,Nikola Jokic,DEN,2023-24,#1d428a
2281,22023,F,0.484340,1630581,Josh Giddey,80,389:35,1991.8,447,2103,67,56,18,179,373,0.480,48,134,0.358,41,52,26,Josh Giddey,OKC,2023-24,#0072CE
2282,22023,G,0.328608,1627777,Georges Niang,82,238:27,1273.4,407,1580,73,35,11,149,325,0.458,48,134,0.358,61,76,36,Georges Niang,CLE,2023-24,#6F263D


In [28]:
#Add matchup min in int format
matchup_totals_year["MIN"]=matchup_totals_year["MATCHUP_MIN"].str.split(":",expand=True)[0]
matchup_totals_year["SEC"]=matchup_totals_year["MATCHUP_MIN"].str.split(":",expand=True)[1]
matchup_totals_year["MIN"]=matchup_totals_year["MIN"].astype(int)
matchup_totals_year["SEC"]=matchup_totals_year["SEC"].astype(int)/60
matchup_totals_year["TOTAL_MIN"]=matchup_totals_year["MIN"]+matchup_totals_year["SEC"]

In [31]:
# category=['POSITION', 'PERCENT_OF_TIME', 'DEF_PLAYER_ID',
    #    'DEF_PLAYER_NAME', 'GP', 'MATCHUP_MIN', 'PARTIAL_POSS', 'PLAYER_PTS',
    #    'TEAM_PTS', 'MATCHUP_AST', 'MATCHUP_TOV', 'MATCHUP_BLK', 'MATCHUP_FGM',
    #    'MATCHUP_FGA', 'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A',
    #    'MATCHUP_FG3_PCT', 'MATCHUP_FTM', 'MATCHUP_FTA', 'SFL', 'PLAYER_NAME','TOTAL_MIN',
    #    'TEAM_ABBREVIATION', 'SEASON']

cat1="TOTAL_MIN"
cat2="PLAYER_PTS"

fig_scatter_scoring = px.scatter(
  matchup_totals_year[matchup_totals_year["POSITION"]=='TOTAL'], 
  x=cat1, 
  y=cat2,
  color='DEF_PLAYER_NAME',
  color_discrete_map=player_color_dict_shot_dist,     
  title="<b>Matchup {} vs. {} For The {} Season</b>".format(cat1,cat2, year),
  template="plotly_white",
  hover_data=['POSITION', 'PERCENT_OF_TIME', 'DEF_PLAYER_ID',
       'DEF_PLAYER_NAME', 'GP', 'MATCHUP_MIN', 'PARTIAL_POSS', 'PLAYER_PTS',
       'TEAM_PTS', 'MATCHUP_AST', 'MATCHUP_TOV', 'MATCHUP_BLK', 'MATCHUP_FGM',
       'MATCHUP_FGA', 'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A',
       'MATCHUP_FG3_PCT', 'MATCHUP_FTM', 'MATCHUP_FTA', 'SFL', 'PLAYER_NAME',
       'TEAM_ABBREVIATION', 'SEASON'],
  height=800, 
  width=1200,
  )

fig_scatter_scoring.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter_scoring.update_traces(marker={'size': 10})

fig_scatter_scoring

In [15]:
category="PLAYER_PTS"

# category=['POSITION', 'PERCENT_OF_TIME', 'DEF_PLAYER_ID',
    #    'DEF_PLAYER_NAME', 'GP', 'MATCHUP_MIN', 'PARTIAL_POSS', 'PLAYER_PTS',
    #    'TEAM_PTS', 'MATCHUP_AST', 'MATCHUP_TOV', 'MATCHUP_BLK', 'MATCHUP_FGM',
    #    'MATCHUP_FGA', 'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A',
    #    'MATCHUP_FG3_PCT', 'MATCHUP_FTM', 'MATCHUP_FTA', 'SFL', 'PLAYER_NAME',
    #    'TEAM_ABBREVIATION', 'SEASON']

In [23]:
fig_matchup_season = px.bar(
  matchup_totals_year[matchup_totals_year["POSITION"]=='TOTAL'].sort_values(by=category, ascending=False).head(25),
  x="DEF_PLAYER_NAME", 
  y=category,
  color="DEF_PLAYER_NAME",
  color_discrete_map=player_color_dict_shot_dist,     
  template='plotly_white',
  title="<b>Defensive Player Matchup {} Totals For The {}</b>".format(category,year),
  hover_data=['POSITION', 'PERCENT_OF_TIME', 'DEF_PLAYER_NAME', 'GP', 'MATCHUP_MIN', 'PARTIAL_POSS', 'PLAYER_PTS',
       'TEAM_PTS', 'MATCHUP_AST', 'MATCHUP_TOV', 'MATCHUP_BLK', 'MATCHUP_FGM',
       'MATCHUP_FGA', 'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A',
       'MATCHUP_FG3_PCT', 'MATCHUP_FTM', 'MATCHUP_FTA', 'SFL', 'PLAYER_NAME',
       'TEAM_ABBREVIATION', 'SEASON'], 
  height=800, 
  width=1200,
  )

fig_matchup_season.update_layout(
    margin=dict(l=5, r=5, t=40, b=5),
    showlegend=False,
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_matchup_season

In [ ]:
fig_matchup_season.write_html("Player {} For The {} Game on {}".format(category,year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOT

In [ ]:
category_multi=['PARTIAL_POSS', 'PLAYER_PTS',
       'TEAM_PTS', 'MATCHUP_AST', 'MATCHUP_TOV', 'MATCHUP_BLK', 'MATCHUP_FGM',
       'MATCHUP_FGA', 'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A',
       'MATCHUP_FG3_PCT', 'MATCHUP_FTM', 'MATCHUP_FTA', 'SFL']
for i in category_multi:
       fig_matchup_season_multi = px.bar(
       matchup_totals_year[matchup_totals_year["POSITION"]=='TOTAL'].sort_values(by=i, ascending=False).head(25),
       x="DEF_PLAYER_NAME", 
       y=i,
       color="DEF_PLAYER_NAME",
       color_discrete_map=player_color_dict_shot_dist,     
       template='plotly_white',
       title="<b>Defensive Player Matchup {} Totals For The {}</b>".format(i,year),
       hover_data=['POSITION', 'PERCENT_OF_TIME', 'DEF_PLAYER_ID',
              'DEF_PLAYER_NAME', 'GP', 'MATCHUP_MIN', 'PARTIAL_POSS', 'PLAYER_PTS',
              'TEAM_PTS', 'MATCHUP_AST', 'MATCHUP_TOV', 'MATCHUP_BLK', 'MATCHUP_FGM',
              'MATCHUP_FGA', 'MATCHUP_FG_PCT', 'MATCHUP_FG3M', 'MATCHUP_FG3A',
              'MATCHUP_FG3_PCT', 'MATCHUP_FTM', 'MATCHUP_FTA', 'SFL', 'PLAYER_NAME',
              'TEAM_ABBREVIATION', 'SEASON'], 
       height=800, 
       width=1200,
       )
       fig_matchup_season_multi.update_layout(
       margin=dict(l=5, r=5, t=40, b=5),
       showlegend=False,
       title_x=0.5,
       hoverlabel=dict(
              bgcolor="white",
              font_size=16,
              font_family="PT Sans Narrow"
       ),
       yaxis = dict(tickfont = dict(size=15)),
       xaxis = dict(tickfont = dict(size=15)),
       font=dict(
              family="PT Sans Narrow",
              size=14,
              color="Black"
       ),
       title_font_family="PT Sans Narrow"
       )
       fig_matchup_season_multi.write_html("Player {} For The {} Game on {}".format(i,year),full_html=False, include_plotlyjs='cdn')